# **1 - Carregamento dos dados tratados**

In [1]:
import pandas as pd

df = pd.read_csv('dados_tratados.csv')
df.head()

,Cliente_Inativo,Sexo,Aposentado,Casado,Dependentes,Meses_Contrato,Assinatura_Telefone,Assinatura_MultiTelefone,Assinatura_Internet,Assinatura_Seguro_online,...,Assinatura_Seguro_dispositivo,Assinatura_Suporte_tecnico,Assinatura_TV,Assinatura_Filmes,Tipo_Contrato,Fatura_Online,Forma_Pagamento,Valor_Mensal,Valor_Diario,Valor_Anual
0,0,0,0,0,0,9,0,0,0,0,...,0,0,0,0,0,0,0,65.6,2.186667,593.30
1,0,1,0,1,1,9,0,1,0,0,...,0,1,1,1,1,1,0,59.9,1.996667,542.40
2,1,1,0,1,1,4,0,0,1,0,...,1,1,1,0,1,0,1,73.9,2.463333,280.85
3,1,1,1,0,1,13,0,0,1,0,...,1,1,0,1,1,0,1,98.0,3.266667,1237.85
4,1,0,1,0,1,3,0,0,1,0,...,0,0,0,0,1,0,0,83.9,2.796667,267.40


# **2 - Análise quantitativo dos cancelamentos**

In [2]:
print('----------------------------------')
print('Quantidade de clientes ativos (No) e inativos (Yes)')
print(df.Cliente_Inativo.value_counts())
print('----------------------------------')
print('Quantidade percentual de clientes ativos (No) e inativos (Yes)')
print(df.Cliente_Inativo.value_counts(normalize=True).map("{:.1%}".format))
print('----------------------------------')

----------------------------------
Quantidade de clientes ativos (No) e inativos (Yes)
0    5163
1    1869
Name: Cliente_Inativo, dtype: int64
----------------------------------
Quantidade percentual de clientes ativos (No) e inativos (Yes)
0    73.4%
1    26.6%
Name: Cliente_Inativo, dtype: object
----------------------------------


In [ ]:
import plotly.express as px

In [ ]:
# Plotando os gráficos relacionais
for coluna in df.columns:
    # para edições nos gráficos: https://plotly.com/python/histograms/
    # para mudar a cor do gráfico , color_discrete_sequence=["blue", "green"]
    
    grafico = px.histogram(df_plot, x=coluna, color="Cliente_Inativo")
    grafico.show()